In [1]:
import pandas as pd
from datetime import datetime
import time
import json
from math import radians, cos, sin, asin, sqrt
from tqdm import tqdm
import numpy as np
tqdm.pandas()

In [2]:
raw = pd.read_csv('../data/HW5/parsed_train_cat.tsv', sep='\t')

In [3]:
raw.columns

Index(['record_number', 'shipment_method_id', 'shipping_fee',
       'carrier_min_estimate', 'carrier_max_estimate', 'category_id',
       'item_price', 'quantity', 'seller_size', 'bt', 'package_size', 'weight',
       'tz_dis', 'dis', 'sender_state', 'receive_state', 'acc_hour',
       'isNextDay', 'pay_hour', 'acc_date', 'isHoliday', 'isCovid', 'target',
       'sm_0', 'sm_1', 'sm_2', 'sm_3', 'sm_4', 'sm_5', 'sm_6', 'sm_7', 'sm_8',
       'sm_9', 'sm_10', 'sm_11', 'sm_12', 'sm_13', 'sm_15', 'ci_0', 'ci_1',
       'ci_2', 'ci_3', 'ci_4', 'ci_5', 'ci_6', 'ci_7', 'ci_8', 'ci_9', 'ci_10',
       'ci_11', 'ci_12', 'ci_13', 'ci_14', 'ci_15', 'ci_16', 'ci_17', 'ci_18',
       'ci_19', 'ci_20', 'ci_21', 'ci_22', 'ci_23', 'ci_24', 'ci_25', 'ci_26',
       'ci_27', 'ci_28', 'ci_29', 'ci_30', 'ci_31', 'ci_32', 'ps_0', 'ps_1',
       'ps_2', 'ps_3', 'ps_4', 'cc_-1.0', 'cc_0.0', 'cc_1.0', 'cs_-1.0',
       'cs_0.0', 'cs_1.0'],
      dtype='object')

In [4]:
raw

,record_number,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,category_id,item_price,quantity,seller_size,bt,...,ps_1,ps_2,ps_3,ps_4,cc_-1.0,cc_0.0,cc_1.0,cs_-1.0,cs_0.0,cs_1.0
0,1,0,-0.525250,0.779787,-0.378132,13,-0.089748,-0.143373,-0.123254,0,...,0,1,0,0,0,0,1,0,0,1
1,3,0,0.262849,0.779787,-0.378132,1,-0.176260,-0.143373,-0.134993,0,...,0,0,0,0,0,0,1,0,0,1
2,6,0,-0.525250,0.779787,-0.378132,3,-0.278462,-0.143373,-0.134993,0,...,0,0,0,0,0,0,1,0,0,1
3,7,0,-0.525250,0.779787,-0.378132,11,-0.328865,-0.143373,-0.134993,0,...,0,0,0,0,0,0,1,0,0,1
4,9,3,-0.525250,-1.273109,2.095049,18,-0.330477,-0.143373,-0.134993,0,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42474,99944,0,0.275109,0.779787,-0.378132,2,-0.357989,-0.143373,-0.134993,1,...,0,0,0,0,0,0,1,0,0,1
42475,99948,0,-0.525250,0.779787,-0.378132,4,-0.327360,-0.143373,-0.111515,0,...,0,1,0,0,0,0,1,0,0,1
42476,99952,0,-0.525250,0.779787,-0.378132,1,-0.250950,-0.143373,-0.134993,0,...,0,0,0,0,0,0,1,0,0,1
42477,99985,1,2.082483,-1.273109,-0.378132,0,0.254580,-0.143373,-0.064560,0,...,0,0,0,0,0,0,1,0,0,1


In [5]:
no_cat = raw.drop(columns=['record_number', 'shipment_method_id', 'category_id', 'package_size', 'sender_state', 'receive_state', 'acc_hour',
       'pay_hour', 'acc_date'])

In [6]:
no_cat.columns

Index(['shipping_fee', 'carrier_min_estimate', 'carrier_max_estimate',
       'item_price', 'quantity', 'seller_size', 'bt', 'weight', 'tz_dis',
       'dis', 'isNextDay', 'isHoliday', 'isCovid', 'target', 'sm_0', 'sm_1',
       'sm_2', 'sm_3', 'sm_4', 'sm_5', 'sm_6', 'sm_7', 'sm_8', 'sm_9', 'sm_10',
       'sm_11', 'sm_12', 'sm_13', 'sm_15', 'ci_0', 'ci_1', 'ci_2', 'ci_3',
       'ci_4', 'ci_5', 'ci_6', 'ci_7', 'ci_8', 'ci_9', 'ci_10', 'ci_11',
       'ci_12', 'ci_13', 'ci_14', 'ci_15', 'ci_16', 'ci_17', 'ci_18', 'ci_19',
       'ci_20', 'ci_21', 'ci_22', 'ci_23', 'ci_24', 'ci_25', 'ci_26', 'ci_27',
       'ci_28', 'ci_29', 'ci_30', 'ci_31', 'ci_32', 'ps_0', 'ps_1', 'ps_2',
       'ps_3', 'ps_4', 'cc_-1.0', 'cc_0.0', 'cc_1.0', 'cs_-1.0', 'cs_0.0',
       'cs_1.0'],
      dtype='object')

In [10]:
res = no_cat.filter(items=['shipping_fee', 'carrier_min_estimate', 'carrier_max_estimate',
       'item_price', 'quantity', 'seller_size', 'bt', 'weight', 'tz_dis',
       'dis', 'isNextDay', 'isHoliday', 'target'], )

In [11]:
res.to_csv("../data/HW5/regre_data.csv", index=None, )

In [18]:
train_np.shape

(42479, 69)

In [25]:
train_label = no_cat['target'].values.reshape(-1, 1)

In [26]:
train_label.shape

(42479, 1)

In [27]:
train = np.concatenate([train_np, train_label], axis=-1)

In [28]:
train.shape

(42479, 70)

In [29]:
train

array([[-0.52524963,  0.77978662, -0.37813177, ...,  0.        ,
         1.        ,  3.        ],
       [ 0.26284947,  0.77978662, -0.37813177, ...,  0.        ,
         1.        ,  3.        ],
       [-0.52524963,  0.77978662, -0.37813177, ...,  0.        ,
         1.        ,  3.        ],
       ...,
       [-0.52524963,  0.77978662, -0.37813177, ...,  0.        ,
         1.        ,  2.        ],
       [ 2.08248273, -1.27310904, -0.37813177, ...,  0.        ,
         1.        ,  2.        ],
       [-0.52524963, -1.27310904, -0.37813177, ...,  0.        ,
         1.        ,  4.        ]])

In [30]:
np.save('../data/HW5/trainset.npy', train)